## ETL de Vendas

### 0. Pré instalações

In [ ]:
!pip install requests
!pip install pandas
!pip install numpy

### 1. Extração

In [ ]:
import requests
import pandas as pd
import numpy as np

In [25]:
url = "https://dummyjson.com/carts"

response = requests.get(url)
if response.status_code != 200:
  raise Exception(f"Erro ao acessar API: {response.status_code}")
else:
  carts = response.json()['carts']

In [ ]:
print("Exemplo de dados extraídos: ")
print(carts[0])

### 2. Transformação

In [ ]:
# Normalização dos dados
extended_data = []
for cart in carts:
    for product in cart['products']:
        extended_data.append({
            'Carrinho_ID': cart['id'],
            'Usuario_ID': cart['userId'],
            'Produto_ID': product['id'],
            'Produto_Nome': product['title'],
            'Quantidade': product['quantity'],
            'Preco_Unitario': product['price'],
            'Total_Item': product['price'] * product['quantity'],
            'Desconto_Total': cart['discountedTotal'],
            'Data_Venda': pd.Timestamp.now().normalize()  # Simulação da data atual
        })

In [ ]:
df_vendas = pd.DataFrame(extended_data)
df_vendas

Transformações Adicionais.

In [17]:
# Correção dos tipos dos dados
df_vendas['Quantidade'] = df_vendas['Quantidade'].astype(int)
df_vendas['Preco_Unitario'] = df_vendas['Preco_Unitario'].astype(float)
df_vendas['Total_Item'] = df_vendas['Total_Item'].astype(float)
df_vendas

,Carrinho_ID,Usuario_ID,Produto_ID,Produto_Nome,Quantidade,Preco_Unitario,Total_Item,Desconto_Total,Data_Venda
0,1,33,168,Charger SXT RWD,3,32999.99,98999.97,89686.65,2025-04-28
1,1,33,78,Apple MacBook Pro 14 Inch Space Grey,2,1999.99,3999.98,89686.65,2025-04-28
2,1,33,183,Green Oval Earring,5,24.99,124.95,89686.65,2025-04-28
3,1,33,100,Apple Airpods,5,129.99,649.95,89686.65,2025-04-28
4,2,142,144,Cricket Helmet,4,44.99,179.96,4288.95,2025-04-28
...,...,...,...,...,...,...,...,...,...
116,29,170,153,Volleyball,5,11.99,59.95,3488.44,2025-04-28
117,30,177,181,Marni Red & Black Suit,1,179.99,179.99,118740.76,2025-04-28
118,30,177,171,Pacifica Touring,4,31999.99,127999.96,118740.76,2025-04-28
119,30,177,35,Potatoes,4,2.29,9.16,118740.76,2025-04-28


In [19]:
# Classificação da compra
df_vendas['Classificacao_Venda'] = np.where(df_vendas['Total_Item'] >= 100, 'Alta', 'Baixa')

In [ ]:
# Criação da Coluna do mês da venda
df_vendas['Mes_Venda'] = df_vendas['Data_Venda'].dt.month

In [22]:
df_vendas.head(10)

,Carrinho_ID,Usuario_ID,Produto_ID,Produto_Nome,Quantidade,Preco_Unitario,Total_Item,Desconto_Total,Data_Venda,Mes_Venda,Classificacao_Venda
0,1,33,168,Charger SXT RWD,3,32999.99,98999.97,89686.65,2025-04-28,4,Alta
1,1,33,78,Apple MacBook Pro 14 Inch Space Grey,2,1999.99,3999.98,89686.65,2025-04-28,4,Alta
2,1,33,183,Green Oval Earring,5,24.99,124.95,89686.65,2025-04-28,4,Alta
3,1,33,100,Apple Airpods,5,129.99,649.95,89686.65,2025-04-28,4,Alta
4,2,142,144,Cricket Helmet,4,44.99,179.96,4288.95,2025-04-28,4,Alta
5,2,142,124,iPhone X,4,899.99,3599.96,4288.95,2025-04-28,4,Alta
6,2,142,148,Golf Ball,4,9.99,39.96,4288.95,2025-04-28,4,Baixa
7,2,142,122,iPhone 6,3,299.99,899.97,4288.95,2025-04-28,4,Alta
8,2,142,110,Selfie Lamp with iPhone,5,14.99,74.95,4288.95,2025-04-28,4,Baixa
9,3,108,98,Rolex Submariner Watch,1,13999.99,13999.99,14144.30,2025-04-28,4,Alta


### 3. Carga

In [23]:
import os

In [24]:
# Exportar para CSV
caminho_arquivo = 'dados_tratados/vendas_dummyjson.csv'
os.makedirs(os.path.dirname(caminho_arquivo), exist_ok=True)
df_vendas.to_csv(caminho_arquivo, index=False)

print(f"Arquivo CSV salvo com sucesso em : {caminho_arquivo}")

Arquivo CSV salvo com sucesso em : dados_tratados/vendas_dummyjson.csv
